In [228]:
import math
import sys

def somme_geometrique(debut, fin, x):
    if x == 1: return fin - debut + 1
    return math.pow(x , debut) * (1 - x ** (fin - debut + 1)) / (1 - x)

class MMS:
    def __init__(self,tauxArrivee,tauxService,s):
        self.s=s
        self.tauxArrivee=tauxArrivee
        self.tauxService=tauxService
        self.u=tauxArrivee/tauxService
        self.rho=self.u/s
        
        #verification de la condition de stabilite du systeme
        
        if self.rho >= 1.0:
            print("le systeme n'est pas stable (le regime stationnaire n'existe pas)")
            exit(-1)
            
        #calcul de p0 
        res = 0.0
        for k in range(0,s):
            res += math.pow(self.u,k) / math.factorial(k)
         
        res += (self.s * math.pow(self.u,s))/(math.factorial(s) * (s - self.u))
        self.p0 = 1/res
        
    # PK
    def pk(self, k):
        if k <= self.s:
            return (math.pow(self.u,k) / math.factorial(k)) * self.p0
        else:
            return (math.pow(self.u,k) / (math.factorial(k) * math.pow(self.s,k-self.s))) * self.p0
    
    # Nf Nombre moyen de client dans la file
    def Nf(self):
        return self.p0 * math.pow(self.u,self.s) * self.rho / math.factorial(self.s) / math.pow((1 - self.rho), 2)
    
    # Ns Nombre moyen de client dans le systeme 
    def Ns(self):
        return self.Nf() + self.tauxArrivee / self.tauxService

    # Tf Temps de sejour moyen dans la file 
    def Tf(self):
        return self.Nf() / self.tauxArrivee
    
    # Ts Temps de sejour moyen dans le systeme 
    def Ts(self):
        return self.Ns() / self.tauxArrivee


class MMSL:
    def __init__(self,tauxArrivee,tauxService,s,capacite):
        self.s=s
        self.tauxArrivee=tauxArrivee
        self.tauxService=tauxService
        self.u=tauxArrivee/tauxService
        self.rho=self.u/s
        self.capacite = capacite
            
        #calcul de p0 
        res = 0.0
        for k in range(0,s):
            res += math.pow(self.u,k) / math.factorial(k)
         
        res += (self.u ** s / math.factorial(s)) * somme_geometrique(0, capacite - s, self.u / s)
        self.p0 = 1/res
        
    # PK
    def pk(self, k):
        if k == 0: return self.p0
        if k > self.capacite: return 0
        if k <= self.s: return math.pow(self.u,k) * self.p0 / math.factorial(k)
        return math.pow(self.u , k) * self.p0 / math.factorial(self.s) / (math.pow(self.s , (k - self.s)))
    
    # Nf
    def Nf(self):
        if self.u == self.s:
            return self.p0 * math.pow(self.s , self.s) * (self.capacite - self.s) * (self.capacite - self.s + 1) / 2 / math.factorial(self.s)
        else:
            return self.p0 * math.pow(self.u , self.s) * self.rho * (1 - math.pow(self.rho , (self.capacite - self.s + 1)) - (1 - self.rho) * (self.capacite - self.s + 1) * math.pow(self.rho , (self.capacite - self.s))) / math.factorial(self.s) / (1 - self.rho) / (1 - self.rho)
    
    # Ns
    def Ns(self):
        return self.Nf() + (self.tauxArrivee * (1 - self.pk(self.capacite))) / self.tauxService

    # Tf
    def Tf(self):
        return self.Nf() / self.tauxArrivee
    
    # Ts
    def Ts(self):
        return self.Ns() / self.tauxArrivee

    

In [229]:
print("systeme MMS")
test = MMS(6,3,3)
print("P0 = " , test.pk(0))
print("nf = " , test.Nf())
print("ns = " , test.Ns())
print("tf = " , test.Tf())
print("ts = " , test.Ts())
#-----------------------------#
print("systeme MMSL")
test1 = MMSL(10,4,3,100)
print("P0 = " , test1.pk(0))
print("nf = " , test1.Nf())
print("ns = " , test1.Ns())
print("tf = " , test1.Tf())
print("ts = " , test1.Ts())

systeme MMS
P0 =  0.1111111111111111
nf =  0.8888888888888886
ns =  2.8888888888888884
tf =  0.1481481481481481
ts =  0.4814814814814814
systeme MMSL
P0 =  0.04494382077349861
nf =  3.511234740264137
ns =  6.011234734158965
tf =  0.3511234740264137
ts =  0.6011234734158964
